In [2]:
from tensorflow import keras
from tensorflow.keras import Sequential, layers

In [4]:
encoder = Sequential([layers.Dense(2, input_shape=[3])])
decoder = Sequential([layers.Dense(3, input_shape=[2])])

autoencoder = Sequential([encoder, decoder])
autoencoder.compile(loss="mse", optimizer="sgd")

In [13]:
import numpy as np
from matplotlib import pyplot as plt
X_train = np.array(np.random.rand(100, 3) * 100, dtype=np.int32)
fig = plt.figure(figsize=(8, 6))
ax = plt.subplot(111, projection='3d')
ax.

TypeError: scatter() missing 1 required positional argument: 'y'